In [1]:
#imports necessary libraries
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize
import os
import random
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import FeatureHasher
from nltk.corpus import stopwords

C:\Anaconda\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.4) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
 
#this below code streams the data from the api

#This to be sent to httpAPI as a parameter
url = 'https://newsapi.org/v2/top-headlines?country=ng&apiKey=002648d343624364a759bffc6ec6c009'

#This function returns news data in json format
def getJsonNewsData(httpAPI):
    response = requests.get(httpAPI)
    print(response)
    jsonNewsData = response.json() #this converts the response to a json format
    return jsonNewsData

In [94]:
def jsonToCsvNormalizer(jsonObject):
    normalized_json = json_normalize(json_object, record_path = ['articles'])
    return normalized_json

In [ ]:
#This function converts the json data and store it in a csv file
#check this later for bug both here and cell 66 of news app
def convAndStoreInCSV(json_object):
    normalized_json = json_normalize(json_object, record_path = ['articles'])
    if(os.path.getsize('news_data.csv') == 0):
        normalized_json.to_csv(r'news_data.csv')
        return True
    else:
        os.remove('news_data.csv') #delete any existing file with the following name then...
        normalized_json.to_csv(r'news_data.csv')#It recreate a file with the name and save normalized_json to it.
        return True

        

In [ ]:
#This might be more efficient than the one above but it seems they do the same thing (to recheck)
#It collects, converts and stores the data in .csv and .json format files 
def collAndStore(jsonObject):
    normalized_json = json_normalize(json_object, record_path = ['articles'])
    
    #saves the data to a csv file
    if(os.path.exists('news_data.csv') == True):
        os.remove('news_data.csv')
        normalized_json.to_csv('news_data.csv')
    elif(os.path.exists('news_data.csv') == False):
        normalized_json.to_csv('news_data.csv')
        
    if(os.path.exists('json_news_data.json') == True):
        os.remove('json_news_data.json')
        pd.read_csv(r'news_data.csv').to_json(r'json_news_data.json') #this converts the file to a json format
    elif(os.path.exists('json_news_data.json') == False):
        pd.read_csv(r'news_data.csv').to_json(r'json_news_data.json') #this converts the file to a json format
        
    the_json_type_data = pd.read_json(r'json_news_data.json')
    print(the_json_type_data)
    
    #here the code to send the data to the phone should be written here! 
    sendDataToPhone(normalized_json)    

In [ ]:
#This function sends the json data to the phone
def sendDataToPhone(data):
    #The code to send the data to the phone goes here!

In [ ]:
#IMPORTANT DOC:
#This function is used for loading previous csv data from file and then appending data to it,
#but the data is in json format
def loadAndAppend(jsonObject):
    #This converts new json to csv data
    new_normalized_csv = jsonToCsvNormalizer(jsonObject)
    
    #this loads and appends the data
    original = pd.read_csv('news_data.csv')
    new_normalized_csv["ID"] = 'NaN' #this creates a necessary column for ID in the new pasrsed in df
    new_normalized_csv["Likes"] = 'NaN' #this creates a necessary column for Likes in the new pasrsed in df
    original.append(new_normalized_csv)
    original.drop(original.columns[0], axis=1) #this removes the previous unwanted indexing
    
    #generates and assign unique IDs for each row of the df
    assignID(original)
    
    #del the prev file
    os.remove('news_data.csv')
    
    #create a new df thus just like overwriting it
    original.to_csv('news_data.csv')
    return True
    

In [54]:
#IMPORTANT DOC:
#Whilst using this function one must pass in atleast an empty list or a list in other to avoid catastrophic error. 

def uniqueIdGen(listOfPrevIDs):
    prevIDs = listOfPrevIDs
    
    #This block of code generates the id
    generator = [random.choice('1234567890'),random.choice('1234567890'),random.choice('1234567890'),random.choice('1234567890'),random.choice('1234567890'),random.choice('1234567890'),random.choice('1234567890'),random.choice('1234567890'),random.choice('1234567890'),random.choice('1234567890'),random.choice('ABCDEF'),random.choice('ABCDEF'),random.choice('ABCDEF'),random.choice('ABCDEF'),random.choice('ABCDEF'),random.choice('ABCDEF')]
    random.shuffle(generator)
    uniqueID = ''.join(generator)
    
    if uniqueID not in prevIDs:
        listOfPrevIDs.append(uniqueID)
        return uniqueID
    else:
        uniqueIdGen(prevIDs)

In [84]:
#original = pd.read_csv('news_data.csv')
#original.append(df)

In [86]:
original["ID"] = 'NaN'
#`original.drop(original.columns[0], axis=1)

In [52]:
#df = pd.read_csv(r'news_data.csv')

In [90]:
#This function genarates and assign the IDs to rows of dataset
def assignID(df):
    if("ID" not in df.columns):
        df["ID"] = 'NaN' #creates a column and assign NaN to it
        for i in df.index:
            df["ID"][i] = uniqueIdGen(df["ID"].to_list())
    elif("ID" in df.columns):
        for i in df.index:
            df["ID"][i] = uniqueIdGen(df["ID"].to_list())
        

In [33]:
#original.drop(original.columns[10], axis = 1, inplace=True)

In [79]:
#original["ID"][0] = uniqueIdGen(original["ID"].to_list()) 

In [ ]:
#creates a column named likes

In [16]:
#df4 = df2.merge(df3, on='source.id')

In [ ]:
def loadAndFeedLikes():
    likesDf = getLikesDf() #This returns the likes and ID df gotten from phone but hasn't been created yet. 
    
    #this loads the data
    original = pd.read_csv('news_data.csv')
    original.drop(original.columns[0], axis=1) #this removes the previous unwanted indexing #But check if this works right!
    
    #merge the DFs
    original.merge(likesDf, on='ID')
    
    #del the prev file
    os.remove('news_data.csv')
    
    #create a new df thus just like overwriting it
    original.to_csv('news_data.csv')
    

In [ ]:
def trainModelAndPredict(toPredict):
    #convert the toPredict data into csv
    toPredictA = jsonToCsvNormalizer(toPredict)
    
    #we need to filter the df thats going to be passed in the model  for prediction i.e removing unnecessary columns
    toPredictB = dfFilter(toPredictA, ['author', 'title', 'description', 'source.name'])
    
    #If there's need to convert toPredict data into array so it could be fit into a clf, it should be done here
    #####
    
    #loads the data
    original = pd.read_csv('news_data.csv')
    original.drop(original.columns[0], axis=1)
    
    #for now i'm going to drop the publishedAt column until we understand how to use time series data well.
    #we would also drop the content untill we know what best to do with it
    #url and urlToImage should also be dropped for now because they wouldnt help in training the model
    #
    
    X = original[['author', 'title', 'description', 'source.name']]
    y = original[['Likes']]
    
    #This function turns the dataset into the model processable data
    readyTheData(X, y)
    
    #Model training
    clf = SVC()
    clf.fit(X, y)
    
    #Prediction of new data
    predLikes = clf.predict(toPredictB)
    
    #Removes previous/latest prediction saved in file other to pass in new prediction in future
    if (os.path.exists('latest_prediction.csv') == True):    #---potential error
        os.remove('latest_prediction.csv')
        
    #save latest prediction in file
    predLikes.to_csv('latest_prediction.csv')
        
    #convert the toPredict and prediction to a new df
    predDf = pd.Dataframe([toPredictB, predLikes], columns=original.columns) #check here for syntax error #and other errors relatedto columns
    
    #convert to json and save in a memory/variable for later usage #check here for syntax error
    #predJson = predDf.to_json()
    
    #merge the saved data to the original data and overwrite the file #check here for syntax error
    original = original.append(predDf)
    original.to_csv('news_data.csv')
    
    #There should be a condition here to check if its not a first time user then send the data
    
        #send the data saved in memory to the phone 
        sendToPhone(predDf)

In [ ]:
def dfFilter(df, columnsWeWantAsList):
    return df[columnsWeWantAsList]

In [ ]:
#There are parts of this code that i do not understand and thus must understand it before moving further
#It will be indicated with #---DNU -meaning do not understand
def encodeAColumn(column):
    for i in column.index:
        toConvert = column[i]
        toConvert = toConvert.encode('utf-8')
        toConvert = int.from_bytes(toConvert, 'little') #---DNU
        column[i] = toConvert
    return True

In [ ]:
def decodeAColumn(column):
    for i in column.index:
        toConvert = column[i]
        toConvert = toConvert.to_bytes((myint.bit_length() + 7) // 8, 'little')
        toConvert = toConvert.decode('utf-8')
        column[i] = toConvert
    return True
        

In [ ]:
def readyTheData(X, y):
    #encode some  columns int asy data for system to interprete
    encodeAColumn(X['author'])
    encodeAColumn(X['source.name'])
    
    #preprocesses and converts those columns into machine readable formats
    columnPreprocessing(X['title'], 'title')
    columnPreprocessing(X['description'], 'description')
    
    #deal with publishedAt here its a time series data
    #for now i'm going to drop the publishedAt column until we understand how to use time series data well.
    

In [ ]:
#This function collects the column unique contents of a df
def collectUniqueContent(column_name):
    listOfUniques = []
    for i in column_name.index:
        if(column_name[i] not in listOfUniques):
            listOfUniques.append(column_name[i])
    return listOfUniques

In [ ]:
#This function should contain a creation of a dataset and a mini model to pass it to 
#Whereby we categorize the data based on if its political, sport, business and entertainment etc.

#The function should collect a column and then extract all the words and turn them into a dataset excluding two and three letter words
#Save to a file and load it back to memory 
#Then train the model using clustering model
#Predict new data
#increment/overwrite the dataset and file
#overwrite the values of the column being passed with that of the labels

def columnPreprocessing(column, name_of_main_column):
    the_column = column
    
    #removes all the stopwords from the sentence of each row
    the_column = removeStopwords(the_column)
    
    #collects and store all unique (every word ones...) words
    unique_words = collUniqueWords(the_column)
    
    #turns those unique_words to columns
    DFCreator(the_column, unique_words)
    
    #assess them against number of appearance each word makes in the main sentence and assign true or false i.e 1 or 0
    assesAndAssign(the_column, name_of_main_column)
    
    #run the columns through a clustering model then assign the values to the  original column 
    clusterModel(the_column, column)

In [ ]:
#Clustering model that does the final assignment
def clusterModel(the_column, originalColumnName):
    KMeans = KMeans(n_clusters = 4)
    KMmodel = KMeans.fit(the_column)
    labels = KMeans.labels_
    
    count = 0
    for i in labels:
        originalColumnName[count] = i
        count += 1

In [ ]:
#asseser and assigner function
def assesAndAssign(df, nameOfMainColumn):
    for i in df.index:
        for word in df[nameOfMainColumn][i].split(' '):
            if word in df.columns:
                df[word][i] = 1

In [ ]:
#Turns unique words into columns
def DFCreator(init_df, listOfWords):
    for(word in listOfWords):
        init_df[word] = 0
    return

In [ ]:
#collects all the unique words and store of each row in a column 
def collUniqueWords(column_name):
    listOfUniques = []
    for i in column_name.index:
        column_words = column_name[i].split(' ')
        for word in column_words:
            if(word not in listOfUniques):
                listOfUniques.append(word)
    return listOfUniques

In [ ]:
#This function removes all the stopwords and returns the removed stopwords column
def removeStopwords(column):
    for i in column.index:
        column[i] = column[i].replace(' %s '% (stopwords.words('english')), ' ')
    return column

In [ ]:
#This function sends data to the phone
#The funtion filters the data to be sent to the phone
#The data is sent in a json format
def sendToPhone(predictedDataframe):
    #create a group of df based on likes
    predictedDataframe = predictedDataframe.groupby(predictedDataframe['Likes'])
    likes = predictedDataframe.get_group('1') #The one could change depending on if 1 is the true or the false
    
    #convert the csv data thats on the memory to json format
    #send(i.e return) the json data to the phone
    return likes.to_json()